In [7]:
import os, re , PyPDF2
from transformers import pipeline

In [8]:
def extract_pdf_text_all_pages(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        full_text = ""
        # Loop through all pages
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            full_text += page.extract_text() + "\n"
        return full_text
def extract_text_from_multiple_pdfs(directory_path):
    all_text = ""
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(directory_path, filename)
            pdf_text = extract_pdf_text_all_pages(pdf_path)
            all_text += f"--- Text from {filename} ---\n" + pdf_text + "\n"
    return all_text


In [9]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

In [10]:
def chunk_text(text, max_length=1000):
    words = text.split()
    chunks = [' '.join(words[i:i + max_length]) for i in range(0, len(words), max_length)]
    return chunks

In [11]:
from transformers import pipeline
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
qa_model = pipeline("question-answering", model=model_name, device=0)  # device=0 for GPU, device=-1 for CPU
def get_answers_from_text(question, context):
    return qa_model(question=question, context=context)
def process_text_in_chunks(question, text_chunks):
    answers = []
    for chunk in text_chunks:
        result = get_answers_from_text(question, chunk)
        answers.append(result['answer'])
    return answers

RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
def save_answers_to_file(answers, output_file="answers.txt"):
    with open(output_file, "w") as file:
        for i, answer in enumerate(answers):
            file.write(f"Answer from chunk {i+1}: {answer}\n")

In [ ]:
def main(directory_path, question):
    all_pdfs_text = extract_text_from_multiple_pdfs(directory_path)
    preprocessed_text = preprocess_text(all_pdfs_text)
    text_chunks = chunk_text(preprocessed_text, max_length=1000)
    answers = process_text_in_chunks(question, text_chunks)
    save_answers_to_file(answers)
    for i, answer in enumerate(answers):
        print(f"Answer from chunk {i+1}: {answer}")
directory_path = './'  # Replace with your PDF folder path
question = "What is ORCA"
main(directory_path, question)

Answer from chunk 1: NEBTS method


# All in single cell

In [ ]:

import os
import re
import PyPDF2
from transformers import pipeline
import torch

def extract_pdf_text_all_pages(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        full_text = ""
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            full_text += page.extract_text() + "\n"
        return full_text
def extract_text_from_multiple_pdfs(directory_path):
    all_text = ""
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(directory_path, filename)
            pdf_text = extract_pdf_text_all_pages(pdf_path)
            all_text += f"--- Text from {filename} ---\n" + pdf_text + "\n"
    return all_text

def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

def chunk_text(text, max_length=1000):
    words = text.split()
    chunks = [' '.join(words[i:i + max_length]) for i in range(0, len(words), max_length)]
    return chunks

device = 0 if torch.cuda.is_available() else -1

qa_model = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", device=device)

def get_answers_from_text(question, context):
    return qa_model(question=question, context=context)
def process_text_in_chunks(question, text_chunks):
    answers = []
    for chunk in text_chunks:
        result = get_answers_from_text(question, chunk)
        answers.append(result['answer'])
    return answers

def main(directory_path, question):
    all_pdfs_text = extract_text_from_multiple_pdfs(directory_path)
    preprocessed_text = preprocess_text(all_pdfs_text)
    text_chunks = chunk_text(preprocessed_text, max_length=1000)
    answers = process_text_in_chunks(question, text_chunks)
    
    for i, answer in enumerate(answers):
        print(f"Answer from chunk {i+1}: {answer}")

directory_path = './'  # Replace with your PDF folder path
question = "What is ORCA "
main(directory_path, question)

Answer from chunk 1: NEBTS method


# Transformers on single paragraph

In [ ]:
from transformers import pipeline
qa_pipeline = pipeline("question-answering")
paragraph = """
For the geometrical optimization of the reactant and product states, and the TS, you should use the B3LYP functional along with the D3 version of Grimme’s dispersion correction with Becke- Johnson damping. You should use the def2-SVP basis set that is of double zeta quality. You should use the SMD solvent model to emulate the experimental conditions. You should employ tight convergence criteria for geometrical optimization and TS search. You can use either the NEB-TS method in ORCA or the QST2+IRC method in Gaussian 16.
"""
questions = [
    "What is ORCA",
    "What details can you extract from the text?",
    "How does the paragraph relate to the topic?",
    # Add more questions as needed
]

for question in questions:
    result = qa_pipeline(question=question, context=paragraph)
    print(f"Question: {question}\nAnswer: {result['answer']}\n")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Question: What is ORCA
Answer: NEB-TS method

Question: What details can you extract from the text?
Answer: QST2+IRC method in Gaussian 16

Question: How does the paragraph relate to the topic?
Answer: double zeta quality



#### collected By : Amar Raj Ghimire

